In [2]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from collections import defaultdict
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
)
from modules.stochastic_program.factory import StochasticProgramFactory



# Prepare Model Parameters
## Scenario Data
Make scenarios complete cross product

In [4]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [5]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [6]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [7]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [8]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [9]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [10]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Other Model Parameters


In [11]:
distances = pd.read_pickle(PATH_DISTANCES)

In [12]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [13]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

# Create & Solve Stochastic Program

In [14]:
vehicle_types = ALL_VEHICLE_TYPES
FLEET_CAPACITY = {
    "kick_scooter": 8000,
    "bicycle": 3000,
    "car": 2000,
}
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df, vehicle_types)
factory.set_initial_allocation(FLEET_CAPACITY)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.06 seconds
_convert_demand finished in 0.10 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.16 seconds
_set_max_demand finished in 0.02 seconds
set_initial_allocation finished in 0.01 seconds


In [15]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [16]:
stochastic_program.relocations_disabled = False

In [17]:
stochastic_program.create_model()

_create_variables finished in 0.92 seconds
_create_objective_function finished in 1.17 seconds
_create_demand_constraints finished in 1.15 seconds
_create_relocation_binary_constraints finished in 0.01 seconds
_create_big_u_sum_constraints finished in 0.15 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.01 seconds
_create_no_refused_demand_constraints finished in 0.01 seconds
_create_max_trips_constraints finished in 0.56 seconds
_create_vehicle_movement_constraints finished in 0.62 seconds
_create_initial_allocation_constraints finished in 0.01 seconds
_create_non_anticipativity_constraints finished in 0.43 seconds
_create_constraints finished in 2.96 seconds
create_model finished in 5.83 seconds


In [18]:
stochastic_program.solve()

Status: Optimal
Optimal Value of Objective Function:  32590.929604332625
Runtime without preprocessing: 5.01 seconds
solve finished in 7.98 seconds


In [19]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 0.55 seconds
get_summary finished in 0.64 seconds


{'status': 'Optimal',
 'objective': 32590.929604332625,
 'solver_runtime': 5.011268694,
 'n_trips_avg': 7750.5,
 'n_unfilled_demand_avg': 1940.0,
 'demand_avg': 9690.5,
 'n_parking_avg': 0.0,
 'n_relocations_avg': 18249.5}

# Benchmarks
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information

In [19]:
capacities = [
    {
        "kick_scooter": 200,
        "bicycle": 100,
        "car": 50,
    },
    # {
    #     "kick_scooter": 1000,
    #     "bicycle": 500,
    #     "car": 250,
    # },{
    #     "kick_scooter": 2000,
    #     "bicycle": 1000,
    #     "car": 500,
    # }
]


In [20]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.27 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.29 seconds
_set_max_demand finished in 0.02 seconds
Status: Optimal
Optimal Value of Objective Function:  4975.74679123742
Runtime without preprocessing: 4.76 seconds
solve finished in 7.45 seconds


Status: Optimal
Optimal Value of Objective Function:  5015.101492694181
Runtime without preprocessing: 4.37 seconds
solve finished in 7.25 seconds


Status: Optimal
Optimal Value of Objective Function:  -380.4050398438044
Runtime without preprocessing: 3.29 seconds
solve finished in 4.91 seconds


Status: Optimal
Optimal Value of Objective Function:  -360.28799928393386
Runtime without preprocessing: 3.31 seconds
solve finished in 4.96 seconds




In [21]:
results_df = pd.DataFrame.from_dict(results)

In [22]:
results_df

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,4975.746791,4.757992,880,18501,0,520,200,100,50,False,False
1,Optimal,5015.101493,4.366536,881,18500,0,519,200,100,50,False,True
2,Optimal,-380.405040,3.288325,429,18952,971,0,200,100,50,True,False
3,Optimal,-360.287999,3.311330,429,18952,971,0,200,100,50,True,True


In [23]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark

In [24]:
last_demand = scenarios.copy()

last_demand_unstacked = last_demand.unstack("vehicle_types")
last_demand_unstacked[("demand", "car")] = (
    last_demand_unstacked[("demand", "kick_scooter")]
    + last_demand_unstacked[("demand", "car")]
    + last_demand_unstacked[("demand", "bicycle")]
)
last_demand_unstacked
last_demand: pd.DataFrame = (
    last_demand_unstacked["demand"]["car"].to_frame().stack().to_frame(name="demand")
)
last_demand.index = last_demand.index.set_names(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)


In [25]:
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

results = []

for vehicle_types in [["car"], ["bicycle"], ["kick_scooter"]]:
    current_vehicle_type = vehicle_types[0]
    current_fleet_capacity = {
        current_vehicle_type: FLEET_CAPACITY[current_vehicle_type]
    }

    last_demand.index = last_demand.index.set_levels(
        last_demand.index.get_level_values("vehicle_types").map(lambda x: current_vehicle_type),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        last_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


Status: Optimal
Optimal Value of Objective Function:  1053.2134264412337
Runtime without preprocessing: 1.38 seconds
solve finished in 2.13 seconds
Status: Optimal
Optimal Value of Objective Function:  531.7147564579955
Runtime without preprocessing: 1.45 seconds
solve finished in 2.39 seconds
Status: Optimal
Optimal Value of Objective Function:  3726.522039948123
Runtime without preprocessing: 1.54 seconds
solve finished in 2.29 seconds


In [26]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(FLEET_CAPACITY)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  4975.74679123742
Runtime without preprocessing: 4.76 seconds
solve finished in 7.39 seconds


In [27]:
results = pd.DataFrame.from_dict(results)

In [28]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,4975.746791,4.755951,880,18501,0,520,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,5311.450223,4.375853,872,56767,0,528,['car']['bicycle']['kick_scooter']


## Value Of The Stochastic Solution

In [29]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [30]:
results = []
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}

In [31]:
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(FLEET_CAPACITY)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  2490.2406452638033
Runtime without preprocessing: 2.03 seconds
solve finished in 3.35 seconds
Status: Optimal
Optimal Value of Objective Function:  2596.0516856397517
Runtime without preprocessing: 2.02 seconds
solve finished in 3.37 seconds


In [32]:
results = pd.DataFrame.from_dict(results)

In [33]:
results

,status,objective,solver_runtime,n_trips,n_unfilled_demand,n_parking,n_relocations,vehicle_types
0,Optimal,2490.240645,2.025330,431,9111.0,0,269,"[kick_scooter, bicycle, car]"
1,Optimal,2596.051686,2.024955,431,9408.0,0,269,"[kick_scooter, bicycle, car]"
